# Metodología DiD de CodeInterpreter

# Librerías

In [72]:
# !pip install pycountry 
# !pip install countryinfo 
# !pip install altair

In [40]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import pycountry
from countryinfo import CountryInfo as CInfo # Se conecta con la informacion de cada país
import matplotlib.pyplot as plt
import  altair  as  alt # Graficos de promedios
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns

# Read Data

In [30]:
data = pd.read_csv("https://raw.githubusercontent.com/github/innovationgraph/main/data/languages.csv", delimiter=',')
data

,num_pushers,language,language_type,iso2_code,year,quarter
0,2066,HTML,markup,AE,2020,1
1,1627,CSS,markup,AE,2020,1
2,288,Jupyter Notebook,markup,AE,2020,1
3,108,Vue,markup,AE,2020,1
4,1734,JavaScript,programming,AE,2020,1
...,...,...,...,...,...,...
104437,198,TypeScript,programming,ZW,2023,4
104438,150,C,programming,ZW,2023,4
104439,140,Dockerfile,programming,ZW,2023,4
104440,140,PHP,programming,ZW,2023,4


In [31]:
# Filtro de EU
data = data[data.iso2_code != "EU"]

# Filtro de XK = Kosovo
data = data[data.iso2_code != "XK"]

In [32]:
# This will give you the count of rows with NaN values.
nan_rows_count = data.isna().any(axis=1).sum()
print(f"There are {nan_rows_count} rows with NaN values in the dataset.")

There are 35 rows with NaN values in the dataset.


In [33]:
data[data["iso2_code"].isnull()] = "NA"

## Keep the 10 most used programming languages

In [35]:
# Group the data by 'language' and sum 'num_pushers' for each language
data['num_pushers'] = pd.to_numeric(data['num_pushers'], errors='coerce')

# Step 2: Fill NaN values with 0 (optional, based on your context)
# data['num_pushers'].fillna(0, inplace=True)
language_usage = data.groupby('language')['num_pushers'].sum()
language_list = language_usage.sort_values(ascending=False).index.tolist()
# # Display the sorted result
# print(language_usage)
# Convert the language usage Series to a list of languages sorted by frequency of use
top_program_lang = language_list[:]
top_program_lang

['HTML',
 'JavaScript',
 'CSS',
 'Python',
 'Shell',
 'Java',
 'TypeScript',
 'Dockerfile',
 'SCSS',
 'C++',
 'C',
 'Makefile',
 'PHP',
 'Jupyter Notebook',
 'Ruby',
 'C#',
 'Batchfile',
 'CMake',
 'Objective-C',
 'Kotlin',
 'Go',
 'Vue',
 'Swift',
 'PowerShell',
 'Assembly',
 'Hack',
 'Dart',
 'Perl',
 'R',
 'TeX',
 'Roff',
 'Less',
 'Lua',
 'Rust',
 'Blade',
 'Vim Script',
 'Procfile',
 'Smarty',
 'EJS',
 'TSQL',
 'HCL',
 'HLSL',
 'ShaderLab',
 'MATLAB',
 'XSLT',
 'Handlebars',
 'M4',
 'GLSL',
 'Starlark',
 'Objective-C++',
 'Groovy',
 'Gherkin',
 'Emacs Lisp',
 'PLpgSQL',
 'Scala',
 'Solidity',
 'Nix',
 'Awk',
 'Pug',
 'Yacc',
 'Jinja',
 'Rich Text Format',
 'ASP.NET',
 'Lex',
 'Mustache',
 'Cuda',
 'Fortran',
 'QMake',
 'CoffeeScript',
 'Tcl',
 'Sass',
 'Haskell',
 'sed',
 'Svelte',
 'Pascal',
 'Cython',
 'Mako',
 'Twig',
 'Smalltalk',
 'PLSQL',
 'Clojure',
 'Raku',
 'GDB',
 'FreeMarker',
 'Mathematica',
 'Julia',
 'NSIS',
 'Scheme',
 'ANTLR',
 'NASL',
 'Stylus',
 'Elixir',
 'GAP',

In [36]:
# language_list

In [37]:
data2 = data[data['language'].isin(top_program_lang)]
data2 = data2.reset_index(drop=True)
data2

,num_pushers,language,language_type,iso2_code,year,quarter
0,2066.0,HTML,markup,AE,2020,1
1,1627.0,CSS,markup,AE,2020,1
2,288.0,Jupyter Notebook,markup,AE,2020,1
3,108.0,Vue,markup,AE,2020,1
4,1734.0,JavaScript,programming,AE,2020,1
...,...,...,...,...,...,...
101162,198.0,TypeScript,programming,ZW,2023,4
101163,150.0,C,programming,ZW,2023,4
101164,140.0,Dockerfile,programming,ZW,2023,4
101165,140.0,PHP,programming,ZW,2023,4


In [38]:
data2['year_quarter'] = data2['year'].astype(str) + '-Q' + data2['quarter'].astype(str)
data2
# Reseteamos el index
data2 = data2.reset_index(drop=True)
data2

,num_pushers,language,language_type,iso2_code,year,quarter,year_quarter
0,2066.0,HTML,markup,AE,2020,1,2020-Q1
1,1627.0,CSS,markup,AE,2020,1,2020-Q1
2,288.0,Jupyter Notebook,markup,AE,2020,1,2020-Q1
3,108.0,Vue,markup,AE,2020,1,2020-Q1
4,1734.0,JavaScript,programming,AE,2020,1,2020-Q1
...,...,...,...,...,...,...,...
101162,198.0,TypeScript,programming,ZW,2023,4,2023-Q4
101163,150.0,C,programming,ZW,2023,4,2023-Q4
101164,140.0,Dockerfile,programming,ZW,2023,4,2023-Q4
101165,140.0,PHP,programming,ZW,2023,4,2023-Q4


In [39]:
# Creating a unique identifier
data2['unique_id'] = data2['iso2_code'] + '-' + data2['language']

In [40]:
import pandas as pd
import numpy as np

# Assume 'df' is your original DataFrame after reading the CSV file.
# 'unique_id' and 'year_quarter' are columns in your DataFrame.

# Create a DataFrame of unique identifiers
iso2_code = pd.DataFrame({'iso2_code': data2['iso2_code'].unique()})

# Create a DataFrame of unique identifiers for languages 
language = pd.DataFrame({'language': data2['language'].unique()})

# Create a DataFrame of all time periods
# time_periods = pd.DataFrame({'year_quarter': range(data2['year_quarter'].min(), data2['year_quarter'].max() + 1)})
year_quarter = pd.DataFrame({'year_quarter': data2['year_quarter'].unique()})

# Create the Cartesian product of unique_ids and time_periods
balanced_panel = iso2_code.merge(language, how='cross').merge(year_quarter, how='cross')

balanced_panel["unique_id"] = balanced_panel["iso2_code"] + "-" +balanced_panel["language"]
balanced_panel

# # Merge the balanced panel with the original data
balanced_df = balanced_panel.merge(data2, on=['unique_id', 'year_quarter'], how='left')

# # Merge the DataFrames with suffixes
balanced_df = balanced_panel.merge(data2, on=['unique_id', 'year_quarter'], how='left', suffixes=('', '_y'))

# # Now, drop the columns with '_y' suffix, which are from the right DataFrame
balanced_df = balanced_df.loc[:, ~balanced_df.columns.str.endswith('_y')]
balanced_df 
# # Now 'balanced_df' is a balanced panel with NaNs for missing data
# # You might need to sort the DataFrame depending on your needs
# balanced_df = balanced_df.sort_values(by=['unique_id', 'year_quarter'])


,iso2_code,language,year_quarter,unique_id,num_pushers,language_type,year,quarter
0,AE,HTML,2020-Q1,AE-HTML,2066.0,markup,2020,1
1,AE,HTML,2020-Q2,AE-HTML,3254.0,markup,2020,2
2,AE,HTML,NA-QNA,AE-HTML,NaN,NaN,NaN,NaN
3,AE,HTML,2020-Q3,AE-HTML,3208.0,markup,2020,3
4,AE,HTML,2020-Q4,AE-HTML,3255.0,markup,2020,4
...,...,...,...,...,...,...,...,...
1066864,CG,reStructuredText,2022-Q4,CG-reStructuredText,NaN,NaN,NaN,NaN
1066865,CG,reStructuredText,2023-Q1,CG-reStructuredText,NaN,NaN,NaN,NaN
1066866,CG,reStructuredText,2023-Q2,CG-reStructuredText,NaN,NaN,NaN,NaN
1066867,CG,reStructuredText,2023-Q3,CG-reStructuredText,NaN,NaN,NaN,NaN


In [29]:
# Function to convert quarter format to integer
def quarter_to_int(quarter_string):
    year, q = quarter_string.split('-')
    year = int(year)
    quarter_number = int(q[1])  # Q1, Q2, Q3, Q4 -> 1, 2, 3, 4
    base_year = 2020  # Adjust based on your balanced_df, or set dynamically
    return 4 * (year - base_year) + quarter_number

# Applying the function
balanced_df['quarter'] = balanced_df['year_quarter'].apply(quarter_to_int)
balanced_df['year'] = balanced_df['year_quarter'].str.split('-').str[0]
balanced_df.loc[balanced_df["num_pushers"].isnull(), "num_pushers"] = 0
balanced_df

,iso2_code,language,year_quarter,unique_id,num_pushers,language_type,year,quarter
0,AE,HTML,2020-Q1,AE-HTML,2066.0,markup,2020,1
1,AE,HTML,2020-Q2,AE-HTML,3254.0,markup,2020,2
2,AE,HTML,2020-Q3,AE-HTML,3208.0,markup,2020,3
3,AE,HTML,2020-Q4,AE-HTML,3255.0,markup,2020,4
4,AE,HTML,2021-Q1,AE-HTML,3321.0,markup,2021,5
...,...,...,...,...,...,...,...,...
51835,CG,SCSS,2022-Q4,CG-SCSS,0.0,NaN,2022,12
51836,CG,SCSS,2023-Q1,CG-SCSS,0.0,NaN,2023,13
51837,CG,SCSS,2023-Q2,CG-SCSS,0.0,NaN,2023,14
51838,CG,SCSS,2023-Q3,CG-SCSS,0.0,NaN,2023,15


In [2]:
data

,num_pushers,language,language_type,iso2_code,year,quarter,year_quarter,unique_id
0,2066,HTML,markup,AE,2020,1,2020-Q1,AE-HTML
1,1627,CSS,markup,AE,2020,1,2020-Q1,AE-CSS
2,288,Jupyter Notebook,markup,AE,2020,1,2020-Q1,AE-Jupyter Notebook
3,108,Vue,markup,AE,2020,1,2020-Q1,AE-Vue
4,1734,JavaScript,programming,AE,2020,1,2020-Q1,AE-JavaScript
...,...,...,...,...,...,...,...,...
104437,198,TypeScript,programming,ZW,2023,4,2023-Q4,ZW-TypeScript
104438,150,C,programming,ZW,2023,4,2023-Q4,ZW-C
104439,140,Dockerfile,programming,ZW,2023,4,2023-Q4,ZW-Dockerfile
104440,140,PHP,programming,ZW,2023,4,2023-Q4,ZW-PHP


In [6]:
data.isna().any(axis=1)

0         False
1         False
2         False
3         False
4         False
          ...  
104437    False
104438    False
104439    False
104440    False
104441    False
Length: 101167, dtype: bool

In [9]:
import pandas as pd

# Load the data
url = "https://raw.githubusercontent.com/github/innovationgraph/main/data/languages.csv"

# Load the data, ensuring "NA" is not treated as NaN

# Display the first few rows and the column names
print(data.head())
print(data.columns)

# Explicitly replace NaN in 'iso2_code' with 'NA'
data['iso2_code'].fillna('NA', inplace=True)

# Filter out EU and XK
data = data[(data.iso2_code != "EU") & (data.iso2_code != "XK")]

# Identify rows with NaN values in any column
nan_rows = data[data.isna().any(axis=1)]

# Display the count of rows with NaN values
nan_rows_count = nan_rows.shape[0]
print(f"There are {nan_rows_count} rows with NaN values in the dataset.")

# Display rows with NaN values
nan_rows.head()


   num_pushers          language language_type iso2_code  year  quarter
0         2066              HTML        markup        AE  2020        1
1         1627               CSS        markup        AE  2020        1
2          288  Jupyter Notebook        markup        AE  2020        1
3          108               Vue        markup        AE  2020        1
4         1734        JavaScript   programming        AE  2020        1
Index(['num_pushers', 'language', 'language_type', 'iso2_code', 'year',
       'quarter'],
      dtype='object')
There are 0 rows with NaN values in the dataset.


,num_pushers,language,language_type,iso2_code,year,quarter


In [28]:
import pandas as pd
import numpy as np

# Step 1: Load and preprocess the data
url = "https://raw.githubusercontent.com/github/innovationgraph/main/data/languages.csv"
data = pd.read_csv(url, delimiter=',', keep_default_na=False)

# Step 2: Replace empty strings in 'iso2_code' with 'NA'
data['iso2_code'].fillna('NA', inplace=True)

# Filter out EU and XK
data = data[(data.iso2_code != "EU") & (data.iso2_code != "XK")]

# Step 3: Convert 'num_pushers' to numeric, coerce errors to NaN
data['num_pushers'] = pd.to_numeric(data['num_pushers'], errors='coerce')

# Create 'year_quarter' column
data['year_quarter'] = data['year'].astype(str) + '-Q' + data['quarter'].astype(str)

# Create a unique identifier
data['unique_id'] = data['iso2_code'] + '-' + data['language']

# Drop 'language_type' column if it exists
if 'language_type' in data.columns:
    data = data.drop(columns=['language_type', 'language', 'iso2_code', 'year', 'quarter'])
data

,num_pushers,year_quarter,unique_id
0,2066,2020-Q1,AE-HTML
1,1627,2020-Q1,AE-CSS
2,288,2020-Q1,AE-Jupyter Notebook
3,108,2020-Q1,AE-Vue
4,1734,2020-Q1,AE-JavaScript
...,...,...,...
104437,198,2023-Q4,ZW-TypeScript
104438,150,2023-Q4,ZW-C
104439,140,2023-Q4,ZW-Dockerfile
104440,140,2023-Q4,ZW-PHP


In [30]:
data.unique_id.nunique()

8513

In [42]:
import pandas as pd
import numpy as np

# Step 1: Load and preprocess the data
url = "https://raw.githubusercontent.com/github/innovationgraph/main/data/languages.csv"
data = pd.read_csv(url, delimiter=',', keep_default_na=False)

# Step 2: Replace empty strings in 'iso2_code' with 'NA'
data['iso2_code'].fillna('NA', inplace=True)

# Filter out EU and XK
data = data[(data.iso2_code != "EU") & (data.iso2_code != "XK")]

# Step 3: Convert 'num_pushers' to numeric, coerce errors to NaN
data['num_pushers'] = pd.to_numeric(data['num_pushers'], errors='coerce')

# Create 'year_quarter' column
data['year_quarter'] = data['year'].astype(str) + '-Q' + data['quarter'].astype(str)

# Create a unique identifier
data['unique_id'] = data['iso2_code'] + '-' + data['language']

# Drop unnecessary columns
data = data.drop(columns=['language_type', 'language', 'iso2_code', 'year', 'quarter'], errors='ignore')

# Step 4: Balance the panel data
# Extract unique year_quarters
year_quarters = sorted(data['year_quarter'].unique())

# Create a MultiIndex with all combinations of unique_id and year_quarter
multi_index = pd.MultiIndex.from_product([data['unique_id'].unique(), year_quarters], names=['unique_id', 'year_quarter'])

# Reindex the data to create a balanced panel
balanced_df = data.set_index(['unique_id', 'year_quarter']).reindex(multi_index).reset_index()

# Step 5: Fill missing values in 'num_pushers' with 0
balanced_df['num_pushers'] = balanced_df['num_pushers'].fillna(0)

# Display the balanced panel data
balanced_df


,unique_id,year_quarter,num_pushers
0,AE-HTML,2020-Q1,2066.0
1,AE-HTML,2020-Q2,3254.0
2,AE-HTML,2020-Q3,3208.0
3,AE-HTML,2020-Q4,3255.0
4,AE-HTML,2021-Q1,3321.0
...,...,...,...
136203,YE-TypeScript,2022-Q4,0.0
136204,YE-TypeScript,2023-Q1,0.0
136205,YE-TypeScript,2023-Q2,0.0
136206,YE-TypeScript,2023-Q3,0.0


In [43]:
# Split 'unique_id' into 'iso2_code' and 'language'
balanced_df['iso2_code'] = balanced_df['unique_id'].str.extract(r'^([A-Z]{2})-')[0]
balanced_df['language'] = balanced_df['unique_id'].str.split('-').str[1:].str.join('-')
balanced_df

,unique_id,year_quarter,num_pushers,iso2_code,language
0,AE-HTML,2020-Q1,2066.0,AE,HTML
1,AE-HTML,2020-Q2,3254.0,AE,HTML
2,AE-HTML,2020-Q3,3208.0,AE,HTML
3,AE-HTML,2020-Q4,3255.0,AE,HTML
4,AE-HTML,2021-Q1,3321.0,AE,HTML
...,...,...,...,...,...
136203,YE-TypeScript,2022-Q4,0.0,YE,TypeScript
136204,YE-TypeScript,2023-Q1,0.0,YE,TypeScript
136205,YE-TypeScript,2023-Q2,0.0,YE,TypeScript
136206,YE-TypeScript,2023-Q3,0.0,YE,TypeScript


## Per capita measures

In [44]:
# Definimos una función que convierte los nombres de países en códigos iso2
def country_to_iso2(country_name):
    try:
        # Intentamos obtener el código iso2 del país usando pycountry
        return pycountry.countries.get(name=country_name).alpha_2
    except AttributeError:
        try:
            # Manejamos casos especiales donde el nombre del país no coincide exactamente con la base de datos de pycountry
            special_cases = {
                "Czechia (Czech Republic)": "CZ",
                "Congo (Congo-Brazzaville)": "CG",
                "Holy See": "VA",
                "Timor-Leste (East Timor)": "TL",
                "Ukraine (with certain exceptions)": "UA",
                "Taiwan": "TW",
                "Bolivia": "BO",
                "Tanzania": "TZ",
                "South Korea": "KR",
                "Moldova": "MD",
                "Brunei": "BN"
            }
            return special_cases[country_name]
        except KeyError:
            return None

# Creamos una lista de países y obtenemos sus códigos iso2 usando la función country_to_iso2
gpt_countries_list = [
    "Albania", "Algeria", "Andorra", "Angola", "Antigua and Barbuda", "Argentina", "Armenia", "Australia", "Austria",
    "Azerbaijan", "Bahamas", "Bangladesh", "Barbados", "Belgium", "Belize", "Benin", "Bhutan", "Bolivia",
    "Bosnia and Herzegovina", "Botswana", "Brazil", "Brunei", "Bulgaria", "Burkina Faso", "Cabo Verde", "Canada",
    "Chile", "Colombia", "Comoros", "Congo (Congo-Brazzaville)", "Costa Rica", "Côte d'Ivoire", "Croatia", "Cyprus",
    "Czechia", "Denmark", "Djibouti", "Dominica", "Dominican Republic", "Ecuador", "El Salvador", "Estonia", "Fiji",
    "Finland", "France", "Gabon", "Gambia", "Georgia", "Germany", "Ghana", "Greece", "Grenada", "Guatemala", "Guinea",
    "Guinea-Bissau", "Guyana", "Haiti", "Holy See", "Honduras", "Hungary", "Iceland", "India", "Indonesia", "Iraq",
    "Ireland", "Israel", "Italy", "Jamaica", "Japan", "Jordan", "Kazakhstan", "Kenya", "Kiribati", "Kuwait",
    "Kyrgyzstan", "Latvia", "Lebanon", "Lesotho", "Liberia", "Liechtenstein", "Lithuania", "Luxembourg", "Madagascar",
    "Malawi", "Malaysia", "Maldives", "Mali", "Malta", "Marshall Islands", "Mauritania", "Mauritius", "Mexico",
    "Micronesia", "Moldova", "Monaco", "Mongolia", "Montenegro", "Morocco", "Mozambique", "Myanmar", "Namibia",
    "Nauru", "Nepal", "Netherlands", "New Zealand", "Nicaragua", "Niger", "Nigeria", "North Macedonia", "Norway",
    "Oman", "Pakistan", "Palau", "Palestine, State of", "Panama", "Papua New Guinea", "Paraguay", "Peru", "Philippines",
    "Poland", "Portugal", "Qatar", "Romania", "Rwanda", "Saint Kitts and Nevis", "Saint Lucia",
    "Saint Vincent and the Grenadines", "Samoa", "San Marino", "Sao Tome and Principe", "Saudi Arabia", "Senegal",
    "Serbia", "Seychelles", "Sierra Leone", "Singapore", "Slovakia", "Slovenia", "Solomon Islands", "South Africa",
    "South Korea", "Spain", "Sri Lanka", "Suriname", "Sweden", "Switzerland", "Taiwan", "Tanzania", "Thailand",
    "Timor-Leste", "Togo", "Tonga", "Trinidad and Tobago", "Tunisia", "Turkey", "Tuvalu", "Uganda", "Ukraine",
    "United Arab Emirates", "United Kingdom", "United States", "Uruguay", "Vanuatu", "Zambia"
]

gpt_countries_iso = [country_to_iso2(country) for country in gpt_countries_list]

# Agregamos una nueva columna gpt_available, con valor 1 si el código iso2 está en la lista gpt_countries_iso, y 0 en caso contrario
balanced_df["gpt_available"] = balanced_df["iso2_code"].apply(lambda row: 1 if row in gpt_countries_iso else 0)



In [47]:
countries = balanced_df.iso2_code.unique()

def create_populations_dictionary():
    country_populations = {}
    special_cases = {"MM": 54688774, "PS": 5483450, "ME": 602445, "AD":79824}
    # special_cases = {}
    for country in countries:
        try:
            country_populations.update({country: CInfo(country).info()["population"]})
        except KeyError:
            try:
                fallback_name = pycountry.countries.lookup(country).name
                country_populations.update({country: CInfo(fallback_name).info()["population"]})
            except KeyError:
                print(country)
                country_populations.update({country: special_cases[country]})

    return country_populations

country_populations = create_populations_dictionary()

ME
MM
PS
AD


# Creación de variables "population", "pushes_pc", "time", "Post" & "Treatment"

**population**

In [50]:
balanced_df["population"] = balanced_df["iso2_code"].map(country_populations)

**pushes_pc**

In [51]:
balanced_df["num_pushers_pc"] = (balanced_df["num_pushers"] / balanced_df["population"])*100000

**time**

In [52]:
balanced_df.num_pushers_pc.describe()

count    136208.000000
mean          6.440186
std          21.444340
min           0.000000
25%           0.000000
50%           0.719111
75%           3.518051
max         668.947310
Name: num_pushers_pc, dtype: float64

**Post**



In [54]:
balanced_df

,unique_id,year_quarter,num_pushers,iso2_code,language,gpt_available,population,num_pushers_pc
0,AE-HTML,2020-Q1,2066.0,AE,HTML,1,9446000,21.871692
1,AE-HTML,2020-Q2,3254.0,AE,HTML,1,9446000,34.448444
2,AE-HTML,2020-Q3,3208.0,AE,HTML,1,9446000,33.961465
3,AE-HTML,2020-Q4,3255.0,AE,HTML,1,9446000,34.459030
4,AE-HTML,2021-Q1,3321.0,AE,HTML,1,9446000,35.157739
...,...,...,...,...,...,...,...,...
136203,YE-TypeScript,2022-Q4,0.0,YE,TypeScript,0,25956000,0.000000
136204,YE-TypeScript,2023-Q1,0.0,YE,TypeScript,0,25956000,0.000000
136205,YE-TypeScript,2023-Q2,0.0,YE,TypeScript,0,25956000,0.000000
136206,YE-TypeScript,2023-Q3,0.0,YE,TypeScript,0,25956000,0.000000


In [53]:
balanced_df.year = balanced_df.year.astype(int)

AttributeError: 'DataFrame' object has no attribute 'year'

In [97]:
# Aplicar la condición y crear la columna post_CI
balanced_df['post_CI'] = (balanced_df['year'] >= 2023) & (balanced_df['quarter'] >= 2)
balanced_df['post_CI'] = balanced_df['post_CI'].astype(int)
balanced_df

,iso2_code,language,year_quarter,unique_id,num_pushers,language_type,year,quarter,gpt_available,population,num_pushers_pc,post_CI
0,AE,HTML,2020-Q1,AE-HTML,2066.0,markup,2020,1,1,9446000,21.871692,0
1,AE,HTML,2020-Q2,AE-HTML,3254.0,markup,2020,2,1,9446000,34.448444,0
2,AE,HTML,2020-Q3,AE-HTML,3208.0,markup,2020,3,1,9446000,33.961465,0
3,AE,HTML,2020-Q4,AE-HTML,3255.0,markup,2020,4,1,9446000,34.459030,0
4,AE,HTML,2021-Q1,AE-HTML,3321.0,markup,2021,5,1,9446000,35.157739,0
...,...,...,...,...,...,...,...,...,...,...,...,...
51835,CG,SCSS,2022-Q4,CG-SCSS,0.0,NaN,2022,12,1,4559000,0.000000,0
51836,CG,SCSS,2023-Q1,CG-SCSS,0.0,NaN,2023,13,1,4559000,0.000000,1
51837,CG,SCSS,2023-Q2,CG-SCSS,0.0,NaN,2023,14,1,4559000,0.000000,1
51838,CG,SCSS,2023-Q3,CG-SCSS,0.0,NaN,2023,15,1,4559000,0.000000,1


**Treatment**

In [98]:
balanced_df['python'] = (balanced_df['language'].isin(['Python'])).astype(int)
balanced_df['Treatment'] = balanced_df.post_CI*balanced_df.python
balanced_df

,iso2_code,language,year_quarter,unique_id,num_pushers,language_type,year,quarter,gpt_available,population,num_pushers_pc,post_CI,python,Treatment
0,AE,HTML,2020-Q1,AE-HTML,2066.0,markup,2020,1,1,9446000,21.871692,0,0,0
1,AE,HTML,2020-Q2,AE-HTML,3254.0,markup,2020,2,1,9446000,34.448444,0,0,0
2,AE,HTML,2020-Q3,AE-HTML,3208.0,markup,2020,3,1,9446000,33.961465,0,0,0
3,AE,HTML,2020-Q4,AE-HTML,3255.0,markup,2020,4,1,9446000,34.459030,0,0,0
4,AE,HTML,2021-Q1,AE-HTML,3321.0,markup,2021,5,1,9446000,35.157739,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51835,CG,SCSS,2022-Q4,CG-SCSS,0.0,NaN,2022,12,1,4559000,0.000000,0,0,0
51836,CG,SCSS,2023-Q1,CG-SCSS,0.0,NaN,2023,13,1,4559000,0.000000,1,0,0
51837,CG,SCSS,2023-Q2,CG-SCSS,0.0,NaN,2023,14,1,4559000,0.000000,1,0,0
51838,CG,SCSS,2023-Q3,CG-SCSS,0.0,NaN,2023,15,1,4559000,0.000000,1,0,0


In [99]:
# balanced_df.Treatment.describe()

In [100]:
unique_values = balanced_df['unique_id'].nunique()
print(unique_values)

3240


### Check Variables

In [106]:
balanced_df.to_csv("C:/Users/Alexander/Documents/GitHub/GPT-Impact-GitHub/output/csv/languages_panel_top20.csv")